# Simple Stack Comparison
- Take a pair of Z-stacks from animals expressing functional indicators, with sustained sensory stimuli (i.e., "steady state" functional stacks)
- Pass the paths to the two stacks (assume no alignment is needed)
- Normalize each, to low and high percentile targets
- Take the difference, such that the whole thing scales between -1 to 1
- Visualize on napari (to guide experiments immediately following)
- Save the difference stack as h5, or possible as tiff or movie

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import napari
import colorcet as cc
from scipy.signal import convolve, correlate
import flammkuchen as fl
from pathlib import Path
from tqdm import tqdm

In [21]:
## Convert colormap to use as napari-digestable dict

cet_cmap = cc.cm.CET_D7
colors = np.vstack([cet_cmap(i) for i in range(256)])

new_colormap = {
    'colors': colors,
    'name': cet_cmap.name,
    'interpolation': 'linear'
}

## Sham data

In [3]:
## Generate sham data (for now)

xx, yy, zz = np.meshgrid(np.linspace(-1,1,400), np.linspace(-1,1,200), np.linspace(-1,1,400)) 

stack0 = (np.sin(xx*10 + zz*np.sin(yy*10))) * (np.sqrt(xx**2 + yy**2 + zz**2) < 0.8) + np.random.rand(*xx.shape)*5
stack1 = (np.cos(xx*23 + yy*np.cos(zz*13))) * (np.sqrt(xx**2 + yy**2 + zz**2) < 0.8) + np.random.rand(*xx.shape)*5

del xx, yy, zz

In [4]:
# normalize, take difference
nstack0 = (stack0 - np.percentile(stack0, 1)) / (np.percentile(stack0, 99) - np.percentile(stack0, 1))
nstack1 = (stack1 - np.percentile(stack1, 1)) / (np.percentile(stack1, 99) - np.percentile(stack1, 1))
dstack = nstack0 - nstack1

In [5]:
del stack0, stack1, nstack0, nstack1

## Real Data

In [41]:
# Define Path
base_path = Path(r'Z:\Ryosuke')
protocol = 'E0091_v04b_SSZ'
fish = '20250507_f4'
path = base_path / protocol / fish 

In [42]:
# List Z stack recs
h5_list = list((path / 'zstack').glob('*.h5'))

for i, zs in enumerate(h5_list):
    print(i, zs.name)

0 zstack_191047.h5
1 zstack_191338.h5


In [43]:
ids = (0,1)

In [44]:
# load, normalize and store stocks
stacks = []
for id in ids:
    temp = fl.load(h5_list[id])['stack_4D'][0, :, :, :]
    temp = (temp - np.percentile(temp, 1)) / (np.percentile(temp, 99) - np.percentile(temp, 1))
    stacks.append(temp)

In [45]:
# frame-wise alignment
n_frame = stacks[0].shape[0]
aligned_dstack = []
for z in range(n_frame):
    peak_px_ind = np.argmax(correlate(stacks[0][z, :, :], stacks[1][z, :, :], mode='same', method='fft'))
    y_shift = peak_px_ind//stacks[0].shape[2] - stacks[0].shape[1]//2
    x_shift = peak_px_ind%stacks[0].shape[2]- stacks[0].shape[2]//2
    print(z, y_shift, x_shift)
    aligned_dstack.append(np.roll(stacks[0][z, :, :], (-y_shift, -x_shift), axis=(0,1)) - stacks[1][z, :, :])
aligned_dstack = np.asarray(aligned_dstack)

0 0 0
1 1 0
2 0 0
3 -2 0
4 -2 -1
5 0 0
6 0 0
7 1 1
8 2 1
9 -1 0
10 0 -1
11 0 -1
12 0 0
13 0 0
14 0 0
15 0 0
16 0 -1
17 0 -1
18 0 0
19 0 0
20 0 0
21 0 0
22 0 0
23 0 0
24 0 -1
25 0 -1
26 0 -1
27 0 -1
28 0 -1
29 0 -1
30 0 -1
31 0 -1
32 0 -1
33 0 -1
34 0 0
35 0 -1
36 0 0
37 0 -1
38 0 0
39 0 -1
40 0 0
41 0 0
42 0 0
43 0 0
44 0 0
45 0 -1
46 0 0
47 0 0
48 0 0
49 0 -1
50 0 -1
51 0 0
52 0 0
53 0 0
54 0 0
55 0 0
56 0 0
57 0 1
58 0 0
59 0 0
60 0 0
61 0 0
62 0 0
63 0 0
64 0 0
65 0 1
66 2 0
67 2 0
68 0 0
69 0 -1
70 0 0
71 2 0
72 0 0
73 0 0
74 1 -1
75 2 2
76 0 0
77 0 0
78 2 0
79 0 -1
80 0 -1
81 0 0
82 0 0
83 2 -1
84 0 -1
85 0 0
86 0 -1
87 1 -1
88 1 0
89 0 0
90 0 0
91 2 0
92 0 0
93 0 -1
94 0 0
95 0 0
96 0 0
97 0 0
98 2 0
99 0 0
100 0 0
101 0 0
102 1 0
103 -1 1
104 2 0
105 2 0
106 2 0
107 0 0
108 1 0
109 0 0


In [73]:
aligned_dstack = stacks[0] - stacks[1]

## Processing

In [46]:
# box filtering
box_size = 3
s_dstack = np.asarray([convolve(frame, np.ones((box_size, box_size))/box_size**2, 'same') for frame in aligned_dstack])

In [47]:
max_cont = 1.5
viewer0 = napari.view_image(
                    s_dstack, 
                    colormap=new_colormap,
                    contrast_limits=(-max_cont, max_cont)
          )



#version 110
#ifdef GL_KHR_blend_equation_advanced
#extension GL_ARB_fragment_coord_conventions : enable
#extension GL_KHR_blend_equation_advanced : enable
#endif
#define lowp
#define mediump
#define highp
#line 1
varying highp vec2 uv;uniform sampler2DRect textureSampler;uniform bool swizzle;uniform highp float opacity;void main() {   highp vec4 tmpFragColor = texture2DRect(textureSampler,uv);   tmpFragColor.a *= opacity;   gl_FragColor = swizzle ? tmpFragColor.bgra : tmpFragColor;}
***


In [48]:
diff_file_name = 'diff_' + h5_list[ids[0]].name[7:-3] + '_' + h5_list[ids[1]].name[7:-3] + '.h5'

In [49]:
# save h5
fl.save(path / diff_file_name, {'d_stack': s_dstack})